In [104]:
config_file = '../mmclassification_bowel/work_dirs/resnet18_8xb32_in1k/resnet18_8xb32_in1k.py'
checkpoint_file = '../mmclassification_bowel/work_dirs/resnet18_8xb32_in1k/epoch_50.pth'

In [105]:
import mmcv
from mmcls.apis import inference_model, init_model, show_result_pyplot

# Specify the device, if you cannot use GPU, you can also use CPU 
# by specifying `device='cpu'`.
device = 'cuda:0'
# device = 'cpu'

# Build the model according to the config file and load the checkpoint.
model = init_model(config_file, checkpoint_file, device=device)

load checkpoint from local path: ../mmclassification_bowel/work_dirs/resnet18_8xb32_in1k/epoch_50.pth


---

In [106]:
from glob import glob
import pandas as pd
import numpy as np

In [107]:
test=pd.read_csv("../data/bmc/bmc_cls_test.txt",names=['idx','label'],sep=' ')

In [108]:
val_list = test['idx']
val_list

0      0069.jpg
1      0045.jpg
2      0101.jpg
3      0171.jpg
4      0662.jpg
         ...   
212    0150.jpg
213    0319.jpg
214    0600.jpg
215    0986.jpg
216    0592.jpg
Name: idx, Length: 217, dtype: object

In [109]:
true_label = test

In [110]:
true_label

,idx,label
0,0069.jpg,0
1,0045.jpg,0
2,0101.jpg,0
3,0171.jpg,0
4,0662.jpg,2
...,...,...
212,0150.jpg,0
213,0319.jpg,0
214,0600.jpg,2
215,0986.jpg,2


In [111]:
model.CLASSES=['0','1','2']

In [112]:
img

'0592.jpg'

In [113]:
file_head = "../data/bmc/"

inference = {"idx":[], "answer":[]}
for img in val_list:
    img_array =  mmcv.imread(file_head+img)
    result = inference_model(model, img_array)
    label0 = (result['pred_label']==0).sum()
    label1 = (result['pred_label']==1).sum()
    label2 = (result['pred_label']==2).sum()
    label_list = [label0,label1,label2]
    inference["answer"].append(np.argmax(label_list))
    inference["idx"].append(img)

In [114]:
test = pd.DataFrame.from_dict(inference)

In [115]:
test.columns = ["idx","answer"]

In [116]:
test

,idx,answer
0,0069.jpg,0
1,0045.jpg,0
2,0101.jpg,0
3,0171.jpg,0
4,0662.jpg,2
...,...,...
212,0150.jpg,0
213,0319.jpg,0
214,0600.jpg,2
215,0986.jpg,1


In [117]:
true_label

,idx,label
0,0069.jpg,0
1,0045.jpg,0
2,0101.jpg,0
3,0171.jpg,0
4,0662.jpg,2
...,...,...
212,0150.jpg,0
213,0319.jpg,0
214,0600.jpg,2
215,0986.jpg,2


In [118]:
df_INNER_JOIN = pd.concat([test,true_label['label']], axis=1)
df_INNER_JOIN

,idx,answer,label
0,0069.jpg,0,0
1,0045.jpg,0,0
2,0101.jpg,0,0
3,0171.jpg,0,0
4,0662.jpg,2,2
...,...,...,...
212,0150.jpg,0,0
213,0319.jpg,0,0
214,0600.jpg,2,2
215,0986.jpg,1,2


In [119]:
wrong = df_INNER_JOIN[df_INNER_JOIN.answer != df_INNER_JOIN.label]
wrong

,idx,answer,label
7,0415.jpg,0,2
16,1017.jpg,1,2
23,0527.jpg,1,2
24,0800.jpg,0,1
32,0816.jpg,0,1
35,0339.jpg,2,1
36,0593.jpg,2,1
55,0953.jpg,0,1
69,0405.jpg,2,0
85,0515.jpg,0,1


In [120]:
wrong.groupby("label").count()["answer"] / len(wrong) * 100

label
0     4.0
1    60.0
2    36.0
Name: answer, dtype: float64

In [121]:
wrong[wrong["label"] == 1].groupby("answer").count()["idx"]

answer
0    7
2    8
Name: idx, dtype: int64

In [122]:
len(df_INNER_JOIN[df_INNER_JOIN.answer == df_INNER_JOIN.label]) / len(df_INNER_JOIN)

0.8847926267281107

In [123]:
# Confusion matrix (resnet18)
# 모델을 각각 불러와서 test set을 평가한다.

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

cf = np.zeros((2,2))


predictions= df_INNER_JOIN['answer']
answers= df_INNER_JOIN['label']


cf = confusion_matrix(answers, predictions)

#fscroe macro추가
fscore = f1_score(answers,predictions,average='macro')
acc = accuracy_score(answers,predictions)

print("Accuracy : {:.4f}% ".format(acc*100))
print("f score : {:.4f} ".format(fscore))
print(cf)
print("-----")



Accuracy : 88.4793% 
f score : 0.8724 
[[78  0  1]
 [ 7 39  8]
 [ 4  5 75]]
-----
